In [36]:
import csv
import json
import requests
import os
import pandas as pd
from dotenv import load_dotenv
import re

In [69]:
df = pd.read_csv("products.csv")

In [70]:
unique_brands = df["Brand"].unique()

In [71]:
unique_brands

array(['Anua', 'APLB', 'AXIS-Y', 'Beauty of Joseon', 'COSRX',
       'Dr. Althea', 'Heimish', 'Medicube', 'Numbuzin', 'Pyunkang Yul',
       'SKIN1004', 'TOCOBO'], dtype=object)

In [72]:
unique_products_by_brand = df.drop_duplicates(subset=["Brand"]).head(10)

In [73]:
unique_products_by_brand

,Product name,Brand,Category/type,Ingredients,Size/packaging,Product image URL,Product page URL
0,"Anua – Heartleaf LHA Moisture Peeling Gel, 120ml",Anua,exfoliants;cleanser,Heartleaf (Houttuynia Cordata) Extract;LHA (Li...,120 ml,https://qudobeauty.com/wp-content/uploads/2025...,https://qudobeauty.com/product/anua-heartleaf-...
4,APLB – Glutathione Niacinamide Facial Cleanser...,APLB,cleanser,Glutathione;Niacinamide (Vitamin B3);Panthenol...,80 ml,https://qudobeauty.com/wp-content/uploads/2025...,https://qudobeauty.com/product/aplb-glutathion...
5,AXIS-Y – Quinoa One Step Balanced Gel Cleanser...,AXIS-Y,cleanser,Quinoa Extract;Centella Asiatica Extract;Green...,180 ml,https://qudobeauty.com/wp-content/uploads/2025...,https://qudobeauty.com/product/axis-y-quinoa-o...
7,"Beauty of Joseon – Red Bean Water Gel, 100ml",Beauty of Joseon,exfoliants;cleanser,Red Bean Extract (Haenam Red Bean 44%);Peptide...,100 ml,https://qudobeauty.com/wp-content/uploads/2025...,https://qudobeauty.com/product/beauty-of-joseo...
11,"COSRX – Low-pH Good Morning Cleanser, 150ml",COSRX,cleanser,Betaine Salicylate (BHA-type exfoliant);Tea Tr...,150 ml,https://qudobeauty.com/wp-content/uploads/2025...,https://qudobeauty.com/product/low-ph-good-mor...
12,"Dr. Althea – Pure Grinding Cleansing Balm, 50ml",Dr. Althea,cleanser,Ground Botanical Complex;Jojoba Oil & Plant-Ba...,50 ml,https://qudobeauty.com/wp-content/uploads/2025...,https://qudobeauty.com/product/dr-althea-pure-...
13,"Heimish – All Clean Balm Mandarin, 120ml",Heimish,cleanser,"Mandarin Extract;Botanical Oils (Coconut, Shea...",120 ml,https://qudobeauty.com/wp-content/uploads/2025...,https://qudobeauty.com/product/heimish-all-cle...
15,Medicube – Zero Pore Pad 2.0 70 Pcs,Medicube,exfoliants;cleanser,AHA & BHA acids;niacinamide;panthenol;moisturi...,70 pcs,https://qudobeauty.com/wp-content/uploads/2025...,https://qudobeauty.com/product/medicube-zero-p...
17,Numbuzin – No.4 Pore Zero Peeled Egg Toner Pad...,Numbuzin,cleanser,Anti Sebum P complex (evening primrose extract...,190 ml (70 pads),https://qudobeauty.com/wp-content/uploads/2025...,https://qudobeauty.com/product/numbuzin-no-4-p...
20,Pyunkang Yul – Low PH Pore Deep Cleansing Foam...,Pyunkang Yul,cleanser;under-5,white willow bark extract (natural BHA);centel...,40 ml,https://qudobeauty.com/wp-content/uploads/2025...,https://qudobeauty.com/product/pyunkang-yul-lo...


In [74]:
products_10 = unique_products_by_brand.head(10).to_dict(orient="records")

In [75]:
products_10

[{'Product name': 'Anua – Heartleaf LHA Moisture Peeling Gel, 120ml',
  'Brand': 'Anua',
  'Category/type': 'exfoliants;cleanser',
  'Ingredients': 'Heartleaf (Houttuynia Cordata) Extract;LHA (Lipo Hydroxy Acid);Panthenol (Vitamin B5);Allantoin;Hyaluronic Acid;Centella Asiatica Extract',
  'Size/packaging': '120 ml',
  'Product image URL': 'https://qudobeauty.com/wp-content/uploads/2025/04/anua-us-cleanser-200ml-heartleaf-lha-moisture-peeling-gel-1161189356-300x300.webp',
  'Product page URL': 'https://qudobeauty.com/product/anua-heartleaf-lha-moisture-peeling-gel-120ml/'},
 {'Product name': 'APLB – Glutathione Niacinamide Facial Cleanser, 80ml',
  'Brand': 'APLB',
  'Category/type': 'cleanser',
  'Ingredients': 'Glutathione;Niacinamide (Vitamin B3);Panthenol (Vitamin B5);Hyaluronic Acid;Allantoin;Adenosine',
  'Size/packaging': '80 ml',
  'Product image URL': 'https://qudobeauty.com/wp-content/uploads/2025/04/res_1378ca8c16a4c7bdec29f75ef7e28d36-300x300.avif',
  'Product page URL': 'h

In [76]:
load_dotenv()

True

In [77]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

In [78]:
def google_search(query, num_results=5):
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "key": GOOGLE_API_KEY,
        "cx": GOOGLE_CSE_ID,
        "q": query,
        "num": num_results
    }

    response = requests.get(url, params=params, timeout=10)
    return response.json()

def build_query(product):
    name = product["Product name"]
    brand = product["Brand"]

    return {
        "official_page": f'"{name}" site:{brand.lower()}.com"',
        "origin": f'"{name}" "Made in"'
    }

In [30]:
test_product = products_10[0]
query = build_query(test_product)

In [32]:
query["origin"]

'"Anua – Heartleaf LHA Moisture Peeling Gel, 120ml" "Made in"'

In [33]:
resp = google_search(query["origin"])

In [41]:
results = resp.get("items", [])

In [44]:
results

[{'kind': 'customsearch#result',
  'title': 'ANUA Heartleaf LHA Moisture Peeling Gel-120ml',
  'htmlTitle': '<b>ANUA Heartleaf LHA Moisture Peeling Gel-120ml</b>',
  'link': 'https://www.skinplusbd.com/anua-heartleaf-lha-moisture-peeling-gel-120ml.html',
  'displayLink': 'www.skinplusbd.com',
  'snippet': 'ANUA Heartleaf LHA Moisture Peeling Gel 120ml. ৳1,850.00. ৳1,650.00. Total ... Hydrates and nourishes the skin. Suitable For-All Skin Type. Made in Korea.',
  'htmlSnippet': '<b>ANUA Heartleaf LHA Moisture Peeling Gel 120ml</b>. ৳1,850.00. ৳1,650.00. Total ... Hydrates and nourishes the skin. Suitable For-All Skin Type. <b>Made in</b> Korea.',
  'formattedUrl': 'https://www.skinplusbd.com/anua-heartleaf-lha-moisture-peeling-gel-120m...',
  'htmlFormattedUrl': 'https://www.skinplusbd.com/<b>anua-heartleaf-lha-moisture-peeling-gel-120m</b>...',
  'pagemap': {'cse_thumbnail': [{'src': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcReecTaAYl27zutDzQJfM1pZ30YJcJbL7WhMLfueh7GOMXNe4

In [79]:
def extract_enrichment(results):
    enrichment = {
        "official_product_page": None,
        "country_of_origin": None,
        "additional_description": None
    }

    for item in results:
        link = item.get("link", "")
        snippet = item.get("snippet", "")

        # official page
        enrichment["official_product_page"] = link

        # country of origin
        match = re.search(r"Made in ([A-Za-z ]+)", snippet)
        if match:
            enrichment["country_of_origin"] = match.group(1).strip()
        
        # description
        enrichment["additional_description"] = snippet
        
    return enrichment

In [48]:
extract_enrichment(results)

{'official_product_page': 'https://fashionskincarebd.com/product/anua-heartleaf-lha-moisture-peeling-gel-120ml/',
 'country_of_origin': 'Korea',
 'additional_description': 'Anua Heartleaf LHA Moisture Peeling Gel 120ml. 1,800.00৳ Original price ... Anua Heartleaf LHA Moisture Peeling Gel 120ml (In Bangladesh), Made in Korea.'}

In [80]:
enriched_products = []

for product in products_10:
    query = build_query(product)

    resp = google_search(query["origin"])
    results = resp.get("items", [])

    enrichment = extract_enrichment(results)

    enriched_products.append({
        **product,
        **enrichment,
        "source": "Google"
    })


In [82]:
enriched_df = pd.DataFrame(enriched_products)
enriched_df.to_csv("enriched_products.csv", index=False)